# MATH 484 Linear Programs and Related Problems

MATH 484 Linear Programs and Related Problems

The Pennsylvania State University

Based on Prof. Christopher Byrne's lecture materials and his _Linear Programs I: A Clear, Concise, and Practical Introduction to Linear Programs and Duality Theory with Applications to Return On Investment (ROI)_.

---

## Imports & Environment

In [1]:
import numpy                    as np
import pandas                   as pd

from   datetime import datetime as d
import locale                   as l
import platform                 as p
import sys                      as s

pad = 20
print(f"{'Executed'.upper():<{pad}}: {d.now()}")
print()
print(f"{'Platform'        :<{pad}}: "
      f"{p.mac_ver()[0]} | "
      f"{p.system()} | "
      f"{p.release()} | "
      f"{p.machine()}")
print(f"{''                :<{pad}}: {l.getpreferredencoding()}")
print()
print(f"{'Python'          :<{pad}}: {s.version}")
print(f"{''                :<{pad}}: {s.version_info}")
print(f"{''                :<{pad}}: {p.python_implementation()}")
print()
print(f"{'NumPy'           :<{pad}}: {np.__version__}")
print(f"{'Pandas'          :<{pad}}: {pd.__version__}")

EXECUTED            : 2023-01-18 09:12:50.281421

Platform            : 13.1 | Darwin | 22.2.0 | arm64
                    : UTF-8

Python              : 3.10.8 | packaged by conda-forge | (main, Nov 22 2022, 08:25:13) [Clang 14.0.6 ]
                    : sys.version_info(major=3, minor=10, micro=8, releaselevel='final', serial=0)
                    : CPython

NumPy               : 1.24.0
Pandas              : 1.5.2


---

\begin{align}
\text{max} \,\,\, 20x_1 + 10x_2 & \,\,\, \text{s.t.} \\
30x_1 + 10x_2 &\le 400 \\
 3x_1 +  4x_2 &\le  80 \\
          x_1 &\le  10 \\
          x_2 &\le  40 \\
     x_1, x_2 &\ge   0 \\
\end{align}

---

The goal of an LP is to find the decision that optimizes some objective subject to certain constraints.

---

## 6 Matrix Forms

1

$
\begin{align}
\text{max} \,\,\, & cx+d \\
Ax &\ge b \\
x &\ge 0 \\
\end{align}
$

2

$
\begin{align}
Ax&\le b \\
x&\ge0
\end{align}
$

3

$
\begin{align}
Ax&= b \\
x&\ge0
\end{align}
$

---

1.2 Def - LINEAR FUNCTION

A function $f : \mathbb{R}^n \rightarrow \mathbb{R}$ is linear if $f(ax + y) = af(x) + f(y) \,\,\, \forall{x, y} \in \mathbb{R}^n, \forall{a} \in \mathbb{R}$.

Note that if $f : \mathbb{R}^n \rightarrow \mathbb{R}$ is linear, then $f(\boldsymbol{0}) = 0$.

1.3 Def - AFFINE FUNCTION

A function $g : \mathbb{R}$ is affine if $f(x) = g(x) + c \,\,\, \forall{x} \in \mathbb{R}^n$ where $g$ is linear and $c \in \mathbb{R}$.

In other words, an affine function is just a linear function plus a constant.

1.4 Def - LINEAR CONSTRAINT

A linear constraint uses a linear function $f : \mathbb{R}^n \rightarrow \mathbb{R}$ and a constant $b \in \mathbb{R}$ to limit the values of a variable $x \in \mathbb{R}^n$ by requiring either $f(x) \le b$, $f(x) \ge b$, or $f(x) = b$.

Only weak inequalities are allowed, which makes the solution set closed.

Note that any affine constraint can be rewritten as a linear constraint.

1.5 Def - [LP] LINEAR PROGRAM

An LP is a problem of optimizing an affine function subject to finitely many linear constraints.

The function to be optimized is called the objective function.

Note that any affine objective function $f(x) + c$ attains its max (or min) where $f(x)$ attains its max (or min).

The constant $c$ only changes the optimal value of $f$, not the values of $x$ at which $f$ attains its maximum (or minimum).

1.6 Def - FEASIBLE SET

The feasible set of an LP is the set of all $x \in \mathbb{R}^n$ that satisfy all the constraints of the LP.

The feasible set may also be called the constraint set or the feasible region.

1.7,1.8 Def - OPTIMAL/SOLUTION SET

$\arg\max_{\Gamma}(f) = \{ x \in \Gamma \,\,\, | \,\,\, f(x) \ge f(y) \,\,\, \forall{y} \in \Gamma \}$

$\arg\max_{\Gamma}(f)$ is the set of points at which $f$ attains its maximum over the set $\Gamma$.

$\arg\min_{\Gamma}(f) = \{ x \in \Gamma \,\,\, | \,\,\, f(x) \le f(y) \,\,\, \forall{y} \in \Gamma \}$

$\arg\min_{\Gamma}(f)$ is the set of points at which $f$ attains its minimum over the set $\Gamma$.

If $\Gamma$ is the feasible set of an LP, then the optimal/solution set of the LP is $\argmax_{\Gamma}(f)$, if the LP is to maximize $f$, or $\argmin_{\Gamma}(f)$, if the LP is to minimize $f$.

1.9 Def - OPTIMAL VALUE

The optimal value of a maximization LP is $\text{max}_{\Gamma}(f)$ where $\Gamma$ is the feasible region of the LP.

The optimal value of a minimization LP is $\text{min}_{\Gamma}(f)$ where $\Gamma$ is the feasible region of the LP.

1.10 Def - LP SOLUTION

The solution to an LP consists of the optimal set and the optimal value.

In other words, solving an LP means finding both the optimal set and the optimal value.

1.11 Def - GRADIENT

Let $f : \mathbb{R}^n \rightarrow \mathbb{R}$. The gradient of $f$ is $\nabla{f} = \left( \frac{\partial f}{\partial x_1}, ..., \frac{\partial f}{\partial x_n} \right)$.

Note that if $f$ is linear or affine, then $\nabla{f}$ is constant.

The significance of the vector $\nabla{f}$ is that it points in the direction of the maximum rate of increase of $f$ per unit Euclidean distance change in the variable $x$ as it varies through $\mathbb{R}^n$.

1.12 Def - DIRECTIONAL DERIVATIVE

If $u \in \mathbb{R}^n$ is a nonzero vector and $f : \mathbb{R}^n \rightarrow \mathbb{R}$ is a function, then the directional derivative of $f$ in the direction of $u$ equals $\frac{\nabla{f} \cdot u}{||u||}$.

Note that the sign of the directional derivative does not depend on the magnitude of $u$. So $\nabla{f} \cdot u$ can be used in place of the true directional derivative when only the sign is needed. The sign is enough to tell whether $f$ increases $(\nabla{f} \cdot u \ge 0)$; decreases $(\nabla{f} \cdot u \le 0)$; or remains constant $(\nabla{f} \cdot u = 0)$ when the variable $x$ is changed in the direction of $u$.

Note also that because the gradient of an affine function is constant, the directional derivative in any direction is constant. (This will be relevant for step 2C of the graphical method.)

1.13 Def - LEVEL SET ?????

If $f : \mathbb{R}^n \rightarrow \mathbb{R}$ a level set of $f$ is a subset of $\mathbb{R}^n$ on which $f$ has constant value $f(x) = b$ for some constant $b \in \mathbb{R}$, then $f(x) = b$ will sometimes be used as an abbreviation to refer to the level set $\{ x \in \mathbb{R}^n \,\,\, | \,\,\, f(x) = b \}$. (Whether the reference is to the equation itself or to the solution of the equation should be clear from the context.)

1.14 - ?????

If $f(x) = b$ on a set $S \in \mathbb{R}^n$ and $x, y \in S$ where $u = x - y$, then $\nabla{f} \cdot u = 0$.

That is, a function has zero directional derivative on any level set.

1.14.5 - using gradients to graph lines

The boundary of a linear inequality constraint in $\mathbb{R}^n$ is a hyperplane of dimension $n - 1$.
* In $\mathbb{R}^2$, a hyperplane of dimension $2 - 1 = 1$ is a line.
* In $\mathbb{R}^3$, a hyperplane of dimension $3 - 1 = 2$ is a plane.

The full solution of a linear equality constraint in $\mathbb{R}^n$ is a hyperplane of dimension $n - 1$.

A quick way to graph lines and an easy way to visually check one's graphs of lines in $\mathbb{R}^2$ is that the boundary of a linear constraint $f(x) \le b$ will be the line $f(x) = b$, a line perpendicular to $\nabla{f}$. Furthermore, $f(0) = 0$, so $\nabla{f}$ can be traced from the origin out to the level set $f(x) = b$.

For example,
* the boundary of the constraint $2x_1 + 3x_2 \le 6$ will be perpendicular to $\nabla{f} = (2, 3)$ and pass through the first quadrant with negative slope
* the boundary of the constraint $2x_1 - 3x_2 \le 6$ will be perpendicular to $\nabla{f} = (2, -3)$ and pass through the fourth quadrant with positive slope
* the boundary of the constraint $-2x_1 - 3x_2 \le 6$ will be perpendicular to $\nabla{f} = (-2, -3)$ and pass through the third quadrant with negative slope
* the boundary of the constraint $-2x_1 + 3x_2 \le 6$ will be perpendicular to $\nabla{f} = (-2, 3)$ and pass through the second quadrant with positive slope

Gradients also make it easy to check which side of an inequality is the feasible side (i.e., the side that satisfies the constraint.)
* the feasible side of the boundary of $f(x) \ge b$ is the side that $\nabla{f}$ points to
* the feasible side of the boundary of $f(x) \le b$ is the opposite side from what $\nabla{f}$ points to (i.e., the side $-\nabla{f}$ points to)
* the feasible set of $f(x) = b$ is just the line itself

1.15 - graphing feasible sets



---

objective function: the affine function to optimize
LP: to optimize the objective function subject to finitely many linear constraints

$\Gamma$ feasible set

$\arg\min$

---

## Class Notes

### 01/27 Friday

a point that has a unique representation

$c_1x_1+c_2x_2+...+c_kx_k$ is convex where $c_i\gt 0$ and $\sum c_i=1$

$x_1,...,x_k\in S$

$y_1=\frac{c_1}{c_1+c_2}x_1+\frac{c_2}{c_1+c_2}x_2\in S$

$(c_1+c_2)y+c_3x_3+...+c_kx_k$

convex hull

$\{\sum_{i=1}^k c_ix_i \,\,\, | \,\,\, c \ge 0 \sum x_i = 1\}$ the smallest convex set containing ${x_1,...,x_k}$

### 01/23 MONDAY

roadmap
* convexity (today)
* matrix forms: packaging the LP
* tableau form (the simplex algorithm uses this)
* final forms: infeasible, feasible but unbounded, feasible
* mapping tableau in control variables
* simplex algorithm

$S \sub \mathbb{R}^n$ is convex if $\forall x,y \in S$ and $\alpha\in[0,1], \alpha x+(1-\alpha)y\in S$

convex combination of x and y

linear combination if coeffs are 0 and sum of coeffs are 1

convex combination of $\{x_1,...,x_k\}$ (a set of vectors) is $\sum c_ix_i$ s.t. $c_i \ge 0,i=1,...k$ and $\sum^k_{i=1} c_i=1$

$x_1 \in \mathbb{R}^n,x_2\in\mathbb{R}^n,...,x_k\in\mathbb{R}^n$

$\alpha\le 1\rightarrow 1-\alpha\ge 0$

geometrically a set $S$ is convex iff $\forall x,y\in S$ the line segment connecting x and y is entirely in S

the empty set is convex

$g(x)\ge b$ where $g:\mathbb{R}^n\rightarrow\mathbb{R}$ is linear

x is generic

suppose y and z are feasible i.e. $g(y)\ge b$ and $g(z)\ge b$

y and z are two specific points that pass the constraint

we need to show that $g(\alpha y+(1-\alpha)z)\ge b \forall\alpha\in [0,1]$

g is linear therefore $\alpha g(y)+(1-\alpha)g(z) \ge \alpha b + (1-\alpha)b = b$

$\alpha g(y) \ge \alpha b$

$(1-\alpha)g(z) \ge (1-\alpha)b$

---

## Linear Map/Transformation

A linear map is a map $V\rightarrow W$ between two vector spaces that preserves the operations of vector addition and scalar multiplication.

Let $V,W$ be vector spaces over the same field $K$. A function $f:V\rightarrow W$ is said to be a linear map if for any two vectors $\mathbf{u},\mathbf{v}\in V$ and any scalar $c\in K$ the following two conditions are satisfied:
1. Additivity $f(\mathbf{u}+\mathbf{v})=f(\mathbf{u})+f(\mathbf{v})$
2. Homogeneity of degree 1 (scalar multiplication) $f(c\mathbf{u})=cf(\mathbf{u})$

A linear map is said to be operation preserving becuase it does not matter whether the linear map is applied before or after the operations of addition and scalar multiplication.

A linear map preserves linear combinations: $f(c_1\mathbf{u}_1+...+c_n\mathbf{u}_n)=c_1f(\mathbf{u}_1)+...+c_nf(\mathbf{u}_n)\,\,\,\forall \mathbf{u}_1,...,\mathbf{u}_n\in V,\forall c_1,...,c_n\in K$ by the associativity of the addition operation

$f(\boldsymbol{0}_V)=f(0\mathbf{v})=0f(\mathbf{v})=\boldsymbol{0}_W$

If a linear map is a bijection, then it is called a linear isomorphism.

In the case where $V=W$, then a linear map is called a linear endomorphism.

A linear map $V\rightarrow W$ always maps the origin of $V$ to the origin of $W$.

A linear map $V\rightarrow W$ maps linear subspaces in $V$ onto linear subspaces in $W$ (possibly of lower dimension).

Linear maps can be represented as matrices.

Rotation

Reflection

---

## Affine Transformation

An affine transformation is a geometric transformation that preserves lines and parallelism but not necessarily Euclidean distances and angles.

Affine Transformations
* Scaling
* Reflection
* Rotation

---

## Terms

* Affine Function
* [Affine Space](https://en.wikipedia.org/wiki/Affine_space)
* [Affine Transformation](https://en.wikipedia.org/wiki/Affine_transformation)
* [Antiderivative](https://en.wikipedia.org/wiki/Antiderivative)
* [Calculus](https://en.wikipedia.org/wiki/Calculus)
* Constraint
* [Curl](https://en.wikipedia.org/wiki/Curl_(mathematics))
* [Curve](https://en.wikipedia.org/wiki/Curve)
* [Derivative](https://en.wikipedia.org/wiki/Derivative)
* [Differentiable Curve](https://en.wikipedia.org/wiki/Differentiable_curve)
* [Differentiable Function](https://en.wikipedia.org/wiki/Differentiable_function)
* [Differential Calculus](https://en.wikipedia.org/wiki/Differential_calculus)
* [Divergence](https://en.wikipedia.org/wiki/Divergence)
* [Euclidean Plane](https://en.wikipedia.org/wiki/Euclidean_plane)
* [Euclidean Plane Isometry](https://en.wikipedia.org/wiki/Euclidean_plane_isometry)
* [Euclidean Space](https://en.wikipedia.org/wiki/Euclidean_space)
* [Function](https://en.wikipedia.org/wiki/Function_(mathematics))
* [Fundamental Theorem of Calculus](https://en.wikipedia.org/wiki/Fundamental_theorem_of_calculus)
* [Geometric Transformation](https://en.wikipedia.org/wiki/Geometric_transformation)
* [Glide Reflection](https://en.wikipedia.org/wiki/Glide_reflection)
* [Gradient](https://en.wikipedia.org/wiki/Gradient)
* [Homothety](https://en.wikipedia.org/wiki/Homothety)
* [Infinitesimal](https://en.wikipedia.org/wiki/Infinitesimal)
* [Integral](https://en.wikipedia.org/wiki/Integral)
* [Isometry](https://en.wikipedia.org/wiki/Isometry)
* [Line Integral](https://en.wikipedia.org/wiki/Line_integral)
* [Linear Map/Transformation](https://en.wikipedia.org/wiki/Linear_map)
* Linear Constraint
* Linear Program (LP)
* [Map](https://en.wikipedia.org/wiki/Map_(mathematics))
* Mathematical Model
* [Multiple Integral](https://en.wikipedia.org/wiki/Multiple_integral)
* [Multivariable Calculus](https://en.wikipedia.org/wiki/Multivariable_calculus)
* [Multivariate Function (Function of Several Variables)](https://en.wikipedia.org/wiki/Function_(mathematics)#Multivariate_function)
* Objective Function
* [Plane](https://en.wikipedia.org/wiki/Plane_(geometry))
* [Reflection](https://en.wikipedia.org/wiki/Reflection_(mathematics))
* [Rotation](https://en.wikipedia.org/wiki/Rotation_(mathematics))
* [Rotations and reflections in two dimensions](https://en.wikipedia.org/wiki/Rotations_and_reflections_in_two_dimensions)
* [Scalar Field](https://en.wikipedia.org/wiki/Scalar_field)
* [Scaling](https://en.wikipedia.org/wiki/Scaling_(geometry))
* [Shearing](https://en.wikipedia.org/wiki/Shear_mapping)
* [Similarity](https://en.wikipedia.org/wiki/Similarity_(geometry))
* [Smoothness](https://en.wikipedia.org/wiki/Smoothness)
* [Surface](https://en.wikipedia.org/wiki/Surface_(mathematics))
* [Surface Integral](https://en.wikipedia.org/wiki/Surface_integral)
* [Symmetry Operation](https://en.wikipedia.org/wiki/Symmetry_operation)
* [Translation](https://en.wikipedia.org/wiki/Translation_(geometry))
* [Vector](https://en.wikipedia.org/wiki/Euclidean_vector)
* [Vector Calculus](https://en.wikipedia.org/wiki/Vector_calculus)
* [Vector Field](https://en.wikipedia.org/wiki/Vector_field)
* [Vector-Valued Function](https://en.wikipedia.org/wiki/Vector-valued_function)

---